In [1]:
import numpy as np
import pandas as pd
from joblib import Parallel, delayed
from strategies_indicators import *
from tqdm import tqdm

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 200)

# Set Training Data

In [3]:
# load data
df_bio = pd.read_csv("data/df_bio_nasdaq.csv", header=[0, 1], index_col=0)
print(df_bio.shape)
df_bio.head(1)

(3512, 5928)


A       AADI      ABBV       ABCL      ABCM      ABEO      ABIO       ABOS       ABSI         ABT      ABUS      ABVC      ACAD       ACB      ACER      ACET  \
                      Adj Close  Adj Close Adj Close  Adj Close Adj Close Adj Close Adj Close  Adj Close  Adj Close   Adj Close Adj Close Adj Close Adj Close Adj Close Adj Close Adj Close   
Datetime                                                                                                                                                                                      
2021-09-02 09:30:00  177.570007  28.142599    112.57  16.950001     21.26     32.75    3.0321  17.120001  19.245001  128.509995     3.465      33.5     17.32      7.32      2.69      8.09   

                          ACHC      ACHL      ACHV      ACIU      ACLX       ACOR       ACRS       ACRX       ACST      ACXP       ADAG      ADAP      ADCT      ADIL      ADMA      ADMP       ADPT  \
                     Adj Close Adj Close Adj Close Adj Close Adj Close  Adj Close  Adj Close  Adj Close  Adj Close Adj Close  Adj Close Adj Close Adj Close Adj Close Adj Close Adj Close  Adj Close   
Datetime                                                                                                                                                                                               
2021-09-02 09:30:00  67.290001      6.71       8.2      7.14       NaN  87.700005  17.929001  21.800001  15.780001     5.235  17.629999      4.95     29.15    104.75     1.345      80.5  39.650002   

                         ADTX       ADUS      ADVM      ADXN      AEZS      AFIB      AFMD       AGE      AGEN       AGIO      AGLE      AGRX       AHCO       AIM      AIRS      AKAN      AKBA  \
                    Adj Close  Adj Close Adj Close Adj Close Adj Close Adj Close Adj Close Adj Close Adj Close  Adj Close Adj Close Adj Close  Adj Close Adj Close Adj Close Adj Close Adj Close   
Datetime                                                                                                                                                                                           
2021-09-02 09:30:00    3580.0  92.204803      2.51       NaN     18.16     12.89    7.0979      1.05  6.005986  45.987499     7.324    2340.0  24.690001     2.122       NaN       NaN     3.085   

                         AKRO       AKTX       AKU      AKYA      ALDX       ALEC      ALGS      ALHC      ALIM      ALKS       ALLK       ALLO      ALLR        ALNY      ALPN      ALRN       ALT  \
                    Adj Close  Adj Close Adj Close Adj Close Adj Close  Adj Close Adj Close Adj Close Adj Close Adj Close  Adj Close  Adj Close Adj Close   Adj Close Adj Close Adj Close Adj Close   
Datetime                                                                                                                                                                                              
2021-09-02 09:30:00     24.18  37.001999      2.51    15.485      9.71  28.809999     17.15     18.83      5.72     30.25  92.254997  25.504999       NaN  197.759995    9.6386      23.4     16.08   

                          ALVR      ALXO      ALZN        AMED        AMGN      AMLX      AMPE      AMPH      AMRN      AMRX       AMS      AMTI       ANAB      ANGO       ANIK       ANIP      ANIX  \
                     Adj Close Adj Close Adj Close   Adj Close   Adj Close Adj Close Adj Close Adj Close Adj Close Adj Close Adj Close Adj Close  Adj Close Adj Close  Adj Close  Adj Close Adj Close   
Datetime                                                                                                                                                                                                
2021-09-02 09:30:00  19.860001     70.93      3.32  184.699997  224.934998       NaN     24.75      19.9      5.39     5.505      2.82     31.67  26.110001     29.17  43.029999  30.530001      5.32   

                         ANNX      ANVS      AORT       APLS      APLT       

In [4]:
# fill missing values
df_bio.fillna(method='ffill', inplace=True)
df_bio.fillna(method='bfill', inplace=True)

In [5]:
# find companies
companies = list(df_bio.columns.get_level_values(0).unique())
len(companies)

988

In [6]:
# mean change in price: start to end
changes = [df_bio[company].iloc[-1].Close / df_bio[company].iloc[0].Close for company in companies]
np.mean(changes)

0.4640260360646511

In [7]:
# filter companies
surviving_companies = [company for company in companies if df_bio[company].iloc[-1].Close / df_bio[company].iloc[0].Close > 0.15]
print(len(surviving_companies))
df_bio = df_bio[surviving_companies]
df_bio.shape

665


(3512, 3990)

In [8]:
# mean change in price: start to end
changes = [df_bio[company].iloc[-1].Close / df_bio[company].iloc[0].Close for company in surviving_companies]
np.mean(changes)

0.6566025011135981

# Prepare Data

In [9]:
# example dataframe: vtyx
vtyx = df_bio["VTYX"].copy()
print(vtyx.shape)
vtyx.head(1)

(3512, 6)


,Adj Close,Close,High,Low,Open,Volume
Datetime,,,,,,
2021-09-02 09:30:00,16.0,16.0,16.0,16.0,16.0,0.0


In [10]:
# set target
vtyx["moving_average"] = vtyx["Close"].rolling(window=18).mean()
vtyx['future_moving_average'] = vtyx['moving_average'].shift(-18)
vtyx['target_regression'] = ((vtyx['future_moving_average'] / vtyx['Close']) - 1) * 100
vtyx['target_classification'] = np.where(vtyx['future_moving_average'] / vtyx['Close'] > 1, 1, -1)

# plot targets for vtyx stock
vtyx[["Close", "future_moving_average", "target_regression", "target_classification"]].iplot()

In [ ]:
# prepare training data
def incorporate_indicators(company):
    # set company
    df_company = df_bio[company].copy()
    df_company["company"] = company
    #incorporate indicators
    df_company, _, _ = sma_strategy(df_company, 200, 1200)
    df_company, _, _ = ema_strategy(df_company, 150, 1350)
    df_company, _, _ = smaema_strategy(df_company, 275)
    df_company, _, _ = macd_strategy(df_company, 125, 170, 100)
    df_company, _, _ = rsi_strategy(df_company, 105, 35)
    df_company, _, _ = stochastic_oscillator_alternative(df_company, 35, 40)
    df_company, _, _ = bollinger_bands(df_company, 45, 1)
    #df_company, _, _ = fibonacci_retracement(df_company, 420)
    df_company, _, _ = rate_of_change(df_company, 3)
    df_company, _, _ = rate_of_change(df_company, 5)
    df_company, _, _ = williams_r(df_company, 3)
    df_company, _, _ = williams_r(df_company, 5)
    df_company, _, _ = chaikin_money_flow(df_company, 820, -0.05)
    df_company = on_balance_volume(df_company)
    df_company = average_true_range(df_company, 7)
    df_company = average_true_range(df_company, 35)
    df_company = average_true_range(df_company, 70)
    # set target 
    df_company["moving_average"] = df_company["Close"].rolling(window=18).mean()
    df_company['future_moving_average'] = df_company['moving_average'].shift(-18)
    df_company['target_regression'] = ((df_company['future_moving_average'] / df_company['Close']) - 1) * 100
    df_company['target_classification'] = np.where(df_company['future_moving_average'] / df_company['Close'] > 1, 1, -1)
    # fill empty values
    df_company.fillna(method='ffill', inplace=True)
    df_company.fillna(method='bfill', inplace=True)
    return df_company

frames = Parallel()(delayed(incorporate_indicators)(company) for company in tqdm(surviving_companies)) #["TNYA", "VTYX"] / surviving_companies
df = pd.concat(frames).reset_index(drop=True)
df.drop(columns=["moving_average","future_moving_average"], inplace=True)
print(df.shape)
df.head(1)

In [ ]:
# save data
df.to_parquet("data/df_bio_training.csv")

# Training

In [ ]:
# load data
df_bio = pd.read_parquet("data/df_bio_training.csv")
print(df_bio.shape)
df_bio.head(1)